In [ ]:
from netCDF4 import Dataset
import pandas as pd
from scipy.io import netcdf

### Data: metadata (like scale_factor)

In [431]:
f = Dataset('2017-12-01-2018-01-01_Dec.nc')
f.variables['water_u']
# scale_factor: 0.001
# missing_value: -30000

<class 'netCDF4._netCDF4.Variable'>
int16 water_u(time, depth, lat, lon)
    long_name: Eastward Water Velocity
    standard_name: eastward_sea_water_velocity
    units: m/s
    _FillValue: -30000
    missing_value: -30000
    scale_factor: 0.001
    add_offset: 0.0
    NAVO_code: 17
    coordinates: time depth lat lon 
unlimited dimensions: 
current shape = (32, 40, 88, 251)
filling off

In [405]:
missing_value = -30000
scale_factor = 0.001
f.close()

### Read

In [406]:
f = netcdf.netcdf_file('2017-12-01-2018-01-01_Dec.nc', 'r')
print(data.variables.keys())

water_u = f.variables['water_u']
water_v = f.variables['water_v']
time = f.variables['time']
depth = f.variables['depth']
lat = f.variables['lat']
lon = f.variables['lon']
all_var = list(zip([water_u, time, depth, lat, lon, water_v,], data.variables.keys()))
for var, name in all_var:
    print(name, ": ", var[:].shape)
    
print('depth: {} ...'.format(depth[:5]))
water_u_suf = water_u.data[:,0,:,:]
water_v_suf = water_v.data[:,0,:,:]
print(water_u.shape)
print(water_u_suf.shape, 'depth=0')

odict_keys(['water_u', 'time', 'depth', 'lat', 'lon', 'water_v'])


### Reconstructure Data

In [419]:
newdata = []
for t in enumerate(time.data):
    for la in enumerate(lat.data):
        for ln in enumerate(lon.data):
            newdata.append([t[1], la[1], ln[1], 
                            water_u_suf[t[0], la[0], ln[0]], 
                            water_v_suf[t[0], la[0], ln[0]]])
            
df = pd.DataFrame.from_records(newdata, columns=['time', 'lat', 'lon', 'u', 'v'])
df.replace({'u': {missing_value: None},
            'v': {missing_value: None}},inplace = True)
df['u'] = scale_factor * df['u']
df['v'] = scale_factor * df['v']
df.dropna(subset=['u', 'v'], inplace=True)

In [422]:
df.describe()

,time,lat,lon
count,706816.000000,706816.000000,706816.000000
mean,157428.000000,38.520000,144.960000
std,221.594381,2.032143,5.796556
min,157056.000000,35.040001,134.959961
25%,157242.000000,36.780000,139.920044
50%,157428.000000,38.520000,144.959961
75%,157614.000000,40.260001,150.000000
max,157800.000000,42.000000,154.959961


In [427]:
df.head()

,time,lat,lon,u,v
45,157056.0,35.040001,138.560059,-0.009,-0.034
46,157056.0,35.040001,138.640015,-0.014,-0.016
47,157056.0,35.040001,138.719971,-0.005,0.041
53,157056.0,35.040001,139.199951,0.047,-0.152
54,157056.0,35.040001,139.280029,0.209,0.288


In [428]:
df.to_csv('sample.csv')

### Viz 

In [441]:
# import cartoframes as cf
# from cartoframes import BaseMap, Layer, styling

# cc = cf.CartoContext()
# cc.write(df, 'sample_plastics', lnglat=('lon', 'lat'), overwrite=True)

In [438]:
# cc.map(layers=[BaseMap(),
#                Layer('sample_plastics',
#                      color={'column': 'v',
#                             'scheme': styling.tealRose(7)})],interactive=True)

In [437]:
# cc.map(layers=[BaseMap(),
#                Layer('sample_plastics',
#                      color={'column': 'u',
#                             'scheme': styling.tealRose(7)})],interactive=True)

<br/><br/>